In [1]:
import numpy as np
import h5py
from keras.models import Model
from keras import models
from keras import layers
from keras import Input
from keras import optimizers
from keras.layers import Dropout
from keras.layers import BatchNormalization
import pydot
import graphviz
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model
from keras.models import load_model
channel_model=['B','C','D','E','F']
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7
set_session(tf.Session(config=config))

#channel_model=['B']

for model_idx in channel_model:
    filepath='./wlan_train_set_model_%s.mat' % model_idx
    f=h5py.File(filepath,'r')
    train_data=np.array(f['Train_data'][:])
    cfo_label=np.array(f['Train_cfo_label'][:])/1e3
    filepath_test='D:/논문연구/WLAN/wlan_test_set_CFO_model_{0}_SNR(-5).mat' .format(model_idx)
    f_test=h5py.File(filepath_test,'r')
    valid_data=np.array(f_test['Test_data'][:])
    valid_label=np.array(f_test['Test_data_cfo_label'][:])/1e3
    input_data=Input(shape=(11,864,1))
    x=layers.Conv2D(32,(3,3),padding='same',activation='relu',kernel_initializer='he_normal')(input_data)
    x=layers.MaxPooling2D((1,4),padding='same')(x)
    x_skip1=layers.Conv2D(128,(1,1),strides=(1,2),padding='same',activation='relu',kernel_initializer='he_normal')(x)
    x=layers.Conv2D(64,(3,3),padding='same',activation='relu',kernel_initializer='he_normal')(x)
    x=layers.MaxPooling2D((1,2),padding='same')(x)
    x=layers.Conv2D(128,(3,3),padding='same',activation='relu',kernel_initializer='he_normal')(x)
    x=layers.add([x_skip1,x])
    x=layers.MaxPooling2D((1,4),padding='same')(x)
    x_skip2=layers.Conv2D(512,(1,1),strides=(1,2),padding='same',activation='relu',kernel_initializer='he_normal')(x)
    x=layers.Conv2D(256,(3,3),padding='same',activation='relu',kernel_initializer='he_normal')(x)
    x=layers.MaxPooling2D((1,2),padding='same')(x)
    x=layers.Conv2D(512,(3,3),padding='same',activation='relu',kernel_initializer='he_normal')(x)
    x=layers.add([x_skip2,x])
    x=layers.Flatten()(x)
    #x=layers.Dense(512,activation='relu',kernel_initializer='he_normal')(x)
    x=layers.Dense(1,name='cfo')(x)
    model= Model(input_data,x)
    model.compile(optimizer=optimizers.Adam(lr=0.001),
                loss={'cfo':'mse'},
                loss_weights={'cfo':1},
                metrics={'cfo':'mse'})
    model.fit(train_data,{'cfo':cfo_label},epochs=20,batch_size=256,validation_data=(valid_data,valid_label))
    
    model_name='wlan_CFO_model_%s_v3.h5' % model_idx
    model.save(model_name)

Using TensorFlow backend.




Train on 100000 samples, validate on 50000 samples
Epoch 1/20
100000/100000 [==============================] - 64s 642us/step - loss: 682.3051 - mse: 682.3052 - val_loss: 574.4439 - val_mse: 574.4438
Epoch 2/20
100000/100000 [==============================] - 61s 611us/step - loss: 247.8394 - mse: 247.8393 - val_loss: 437.4211 - val_mse: 437.4210
Epoch 3/20
100000/100000 [==============================] - 59s 591us/step - loss: 195.5015 - mse: 195.5014 - val_loss: 336.6491 - val_mse: 336.6490
Epoch 4/20
100000/100000 [==============================] - 59s 593us/step - loss: 170.3051 - mse: 170.3051 - val_loss: 301.2816 - val_mse: 301.2816
Epoch 5/20
100000/100000 [==============================] - 60s 596us/step - loss: 155.5351 - mse: 155.5351 - val_loss: 298.5330 - val_mse: 298.5331
Epoch 6/20
100000/100000 [==============================] - 59s 589us/step - loss: 142.4166 - mse: 142.4166 - val_loss: 313.4085 - val_mse: 313.4084
Epoch 7/20
100000/100000 [===========================

100000/100000 [==============================] - 57s 573us/step - loss: 42.6783 - mse: 42.6783 - val_loss: 117.6282 - val_mse: 117.6282
Epoch 13/20
100000/100000 [==============================] - 59s 592us/step - loss: 35.4583 - mse: 35.4583 - val_loss: 132.9595 - val_mse: 132.9596
Epoch 14/20
100000/100000 [==============================] - 60s 596us/step - loss: 32.2900 - mse: 32.2900 - val_loss: 117.7067 - val_mse: 117.7067
Epoch 15/20
100000/100000 [==============================] - 59s 593us/step - loss: 30.2507 - mse: 30.2508 - val_loss: 111.6608 - val_mse: 111.6608
Epoch 16/20
100000/100000 [==============================] - 59s 593us/step - loss: 26.6030 - mse: 26.6030 - val_loss: 110.5228 - val_mse: 110.5228
Epoch 17/20
100000/100000 [==============================] - 59s 590us/step - loss: 26.8424 - mse: 26.8424 - val_loss: 106.8873 - val_mse: 106.8873
Epoch 18/20
100000/100000 [==============================] - 61s 613us/step - loss: 24.1567 - mse: 24.1567 - val_loss: 106.4